In [50]:
!pip install pycaret
!pip install MarkupSafe==2.0.1
!pip install numba==0.53

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
%cd /content/drive/MyDrive/데이콘
path = '/content/drive/MyDrive/데이콘'

/content/drive/MyDrive/데이콘


In [53]:
import pandas as pd
train = pd.read_csv(path + '/train.csv')
test = pd.read_csv(path + '/test.csv')
submission = pd.read_csv(path + '/sample_submission.csv')
train.shape

(15000, 70)

In [54]:
train.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


In [55]:
train.columns

Index(['index', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10',
       'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20',
       'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'country', 'introelapse',
       'testelapse', 'surveyelapse', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'age', 'hand', 'religion', 'orientation',
       'voted', 'married', 'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [56]:
print(train.shape)
print(test.shape)
print(submission.shape)

(15000, 70)
(35452, 69)
(35452, 2)


# drop

In [57]:
# 임의로 드랍
test = test.drop(["index","voted","engnat","hand","education","married","urban","country","familysize"], axis=1)
train = train.drop(["index","voted","engnat","hand","education","married","urban","country","familysize"], axis=1)

# 결측치 처리

In [58]:
# 결측치 평균값으로 대체
#평균치로 조정(null값) 정수값으로 출력하기 위해  round로 반올림
train = train.fillna(round(train.mean()))
test = test.fillna(round(test.mean()))
#null값이 있는지 확인
train.isnull().sum()

Q1             0
Q2             0
Q3             0
Q4             0
Q5             0
              ..
age            0
religion       0
orientation    0
ASD            0
nerdiness      0
Length: 61, dtype: int64

# 이상치 대체

In [59]:
#IQR method - 사분위 수를 이용한 이상치 제거
# introelapse testelapse surveyelapse Age  -- 이상치는 최빈값으로 대체

In [60]:
import numpy as np

In [61]:
Q3, Q1 = np.percentile(train["introelapse"], [75 ,25])
IQR1 = Q3 - Q1
IQR1

32.0

In [64]:
train["introelapse"][(Q1-1.5*IQR1 > train["introelapse"])|(Q3+1.5*IQR1 < train["introelapse"])] = 2

In [63]:
train["introelapse"]

0         3
1         5
2         9
3         2
4         3
         ..
14995    12
14996    31
14997    17
14998    14
14999     8
Name: introelapse, Length: 15000, dtype: int64

In [65]:
Q3, Q1 = np.percentile(test["introelapse"], [75 ,25])
IQR2 = Q3 - Q1
IQR2

34.0

In [66]:
test["introelapse"][(Q1-1.5*IQR2 > test["introelapse"])|(Q3+1.5*IQR2 < test["introelapse"])] = 2

In [67]:
test["introelapse"]

0         9
1        18
2         4
3         2
4        24
         ..
35447    10
35448     2
35449    27
35450     3
35451     2
Name: introelapse, Length: 35452, dtype: int64

In [68]:
Q3, Q1 = np.percentile(train["testelapse"], [75 ,25])
IQR1 = Q3 - Q1
IQR1

58.0

In [69]:
train["testelapse"][(Q1-1.5*IQR1 > train["testelapse"])|(Q3+1.5*IQR1 < train["testelapse"])] = 94

In [70]:
train["testelapse"]

0         94
1         85
2        108
3        121
4         94
        ... 
14995    103
14996    106
14997    103
14998     68
14999    182
Name: testelapse, Length: 15000, dtype: int64

In [71]:
Q3, Q1 = np.percentile(test["testelapse"], [75 ,25])
IQR2 = Q3 - Q1
IQR2

58.0

In [72]:
test["testelapse"][(Q1-1.5*IQR2 > test["testelapse"])|(Q3+1.5*IQR2 < test["testelapse"])] = 94

In [73]:
test["testelapse"]

0        114
1        107
2         87
3        155
4        163
        ... 
35447    200
35448    136
35449    121
35450     58
35451     94
Name: testelapse, Length: 35452, dtype: int64

In [74]:
Q3, Q1 = np.percentile(train["surveyelapse"], [75 ,25])
IQR1 = Q3 - Q1
IQR1

91.0

In [75]:
train["surveyelapse"][(Q1-1.5*IQR1 > train["surveyelapse"])|(Q3+1.5*IQR1 < train["surveyelapse"])] = 145

In [76]:
train["surveyelapse"]

0          6
1        120
2        100
3        139
4        216
        ... 
14995    161
14996    179
14997    168
14998    109
14999    348
Name: surveyelapse, Length: 15000, dtype: int64

In [77]:
Q3, Q1 = np.percentile(test["surveyelapse"], [75 ,25])
IQR2 = Q3 - Q1
IQR2

89.25

In [78]:
test["surveyelapse"][(Q1-1.5*IQR2 > test["surveyelapse"])|(Q3+1.5*IQR2 < test["surveyelapse"])] = 145

In [79]:
test["surveyelapse"]

0        243
1        206
2        191
3        258
4        289
        ... 
35447    254
35448    155
35449    191
35450    105
35451    178
Name: surveyelapse, Length: 35452, dtype: int64

In [80]:
Q3, Q1 = np.percentile(train["age"], [75 ,25])
IQR1 = Q3 - Q1
IQR1

10.0

In [81]:
error = train["age"][(Q1-1.5*IQR1 > train["age"])|(Q3+1.5*IQR1 < train["age"])]
for i in error :
    if i > 100 :
        i = 18

In [82]:
train["age"]

0        20
1        49
2        43
3        17
4        18
         ..
14995    17
14996    45
14997    20
14998    29
14999    21
Name: age, Length: 15000, dtype: int64

In [83]:
Q3, Q1 = np.percentile(test["age"], [75 ,25])
IQR2 = Q3 - Q1
IQR2

10.0

In [84]:
error2 = test["age"][(Q1-1.5*IQR2 > test["age"])|(Q3+1.5*IQR2 < test["age"])]
for i in error2 :
    if i > 100 :
        i = 18

In [85]:
test["age"]

0        19
1        33
2        13
3        28
4        15
         ..
35447    16
35448    16
35449    31
35450    19
35451    14
Name: age, Length: 35452, dtype: int64

# autoML 모델

In [86]:
import jinja2
from pycaret.classification import *

In [87]:
clf = setup(data = train, target = 'nerdiness')

,Description,Value
0,session_id,1731
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 61)"
5,Missing Values,False
6,Numeric Features,43
7,Categorical Features,17
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='nerdiness',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [88]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7691,0.8566,0.8257,0.7717,0.7977,0.5296,0.5315,1.609
rf,Random Forest Classifier,0.7670,0.8519,0.8191,0.7721,0.7948,0.5259,0.5273,1.697
lightgbm,Light Gradient Boosting Machine,0.7462,0.8154,0.8015,0.7536,0.7767,0.4833,0.4847,0.437
gbc,Gradient Boosting Classifier,0.7295,0.8027,0.7892,0.7382,0.7627,0.4490,0.4506,2.504
lda,Linear Discriminant Analysis,0.7235,0.7937,0.7973,0.7272,0.7606,0.4350,0.4378,0.138
ada,Ada Boost Classifier,0.7206,0.7887,0.7740,0.7337,0.7533,0.4319,0.4328,0.652
lr,Logistic Regression,0.7170,0.7836,0.7864,0.7239,0.7538,0.4225,0.4247,2.226
nb,Naive Bayes,0.6702,0.7448,0.6135,0.7449,0.6711,0.3464,0.3546,0.038
qda,Quadratic Discriminant Analysis,0.6229,0.7215,0.4429,0.7899,0.5574,0.2736,0.3155,0.061
dt,Decision Tree Classifier,0.6942,0.6906,0.7258,0.7210,0.7233,0.3815,0.3816,0.155


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=1731, verbose=0,
                     warm_start=False), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
 

In [89]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7733,0.8525,0.8304,0.7742,0.8013,0.5382,0.5400
1,0.7438,0.8345,0.8099,0.7466,0.7769,0.4772,0.4794
2,0.7724,0.8526,0.8332,0.7718,0.8013,0.5358,0.5379
3,0.7714,0.8556,0.8150,0.7800,0.7971,0.5357,0.5364
4,0.7742,0.8559,0.8235,0.7791,0.8007,0.5407,0.5418
Mean,0.7670,0.8502,0.8224,0.7703,0.7955,0.5255,0.5271
Std,0.0116,0.0080,0.0089,0.0122,0.0094,0.0242,0.0239


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   class_weight=None,
                                                   criterion='gini',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,

In [90]:
pred_holdout = predict_model(blended)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   class_weight=None,
                                                   criterion='gini',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                         

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7785,0.8638,0.8325,0.7849,0.808,0.5468,0.5481


In [91]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   class_weight=None,
                                                   criterion='gini',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                       

In [92]:
predictions = predict_model(final_model, data = test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(bootstrap=False,
                                                   ccp_alpha=0.0,
                                                   class_weight=None,
                                                   criterion='gini',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                         

In [93]:
predictions

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,VCL14,VCL15,VCL16,gender,age,religion,orientation,ASD,Label,Score
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,1,1,1,2.0,19,4.0,4.0,2.0,0,0.7011
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,1,1,1,1.0,33,1.0,5.0,2.0,1,0.8207
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,1,1,1,2.0,13,4.0,5.0,2.0,1,0.8589
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,1,1,1,1.0,28,2.0,2.0,2.0,1,0.6739
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,0,1,1,2.0,15,4.0,5.0,2.0,1,0.8288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,1,0,1,2.0,16,10.0,4.0,2.0,1,0.8765
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,1,1,1,3.0,16,2.0,5.0,2.0,1,0.7662
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,1,1,1,1.0,31,1.0,5.0,2.0,1,0.9068
35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,...,1,1,1,1.0,19,12.0,1.0,2.0,0,0.7717


In [94]:
submission['nerdiness'] = predictions['Label']

In [ ]:
submission.to_csv('submission_ver108.csv', index = False)